# AWS Bedrock Agents with Couchbase Vector Search - Custom Control Approach

This notebook demonstrates the Custom Control approach for implementing AWS Bedrock agents with Couchbase Vector Search. In this approach, the agent returns control to the application for function execution.

We'll implement a multi-agent architecture with specialized agents for different tasks:
- **Researcher Agent**: Searches for relevant documents in the vector store
- **Writer Agent**: Formats and presents the research findings

## Alternative Approaches

This notebook demonstrates the Custom Control approach for AWS Bedrock Agents. For comparison, you might also want to check out the Lambda Approach, which uses AWS Lambda functions to execute agent tools instead of handling them directly in your application code.

The Lambda approach offers better separation of concerns and scalability, but requires more setup. You can find that implementation here: [Lambda Approach Notebook](../lambda-approach/Bedrock_Agents_Lambda.ipynb)

Note: If the link above doesn't work in your Jupyter environment, you can navigate to the file manually in the `awsbedrock-agents/lambda-approach/` directory.

## Overview

The Custom Control approach gives the application invoking the agent the responsibility of executing the agent's defined functions (tools). When the agent decides to use a tool, it sends a `returnControl` event back to the calling application, which then executes the function locally and (optionally) returns the result to the agent to continue processing.

## Key Steps & Concepts

1.  **Define Agent:**
    *   Define instructions (prompt) for the agent.
    *   Define the function schema (tools the agent can use, e.g., `researcher_functions`, `writer_functions` in the example).

2.  **Create Agent in Bedrock:**
    *   Use `bedrock_agent_client.create_agent` to create the agent, providing the instructions and foundation model.
    *   The example's `create_agent` function includes logic to check for existing agents and potentially delete/recreate them if they are in a non-functional state.

3.  **Create Action Group (Custom Control):**
    *   Use `bedrock_agent_client.create_agent_action_group`.
    *   Crucially, set the `actionGroupExecutor` to `{"customControl": "RETURN_CONTROL"}`. This tells Bedrock to pause execution and return control to the caller when a function in this group needs to be run.
    *   Provide the `functionSchema` defined earlier.

4.  **Prepare Agent:**
    *   Use `bedrock_agent_client.prepare_agent` to make the agent ready for invocation.
    *   The `wait_for_agent_status` utility function polls until the agent reaches a `PREPARED` or `Available` state.

5.  **Create Agent Alias:**
    *   An alias (e.g., "v1") is created using `bedrock_agent_client.create_agent_alias` for invoking the agent.

6.  **Invoke Agent & Handle Return Control:**
    *   Use `bedrock_runtime_client.invoke_agent`, providing the `agentId`, `agentAliasId`, `sessionId`, and `inputText`. Set `enableTrace=True` for debugging.
    *   Process the streaming response (`response['completion']`).
    *   **Listen for `returnControl` events:** When the agent decides to use a function, the stream will contain an event with a `returnControl` key.
    *   **Parse the Invocation:** Extract the `functionInvocationInput` from the `returnControl` event to get the `actionGroup`, `function` name, and `parameters`.
    *   **Execute the Function Locally:** The example's `invoke_agent` function directly calls the corresponding Python function (e.g., `search_documents` imported from `utils`, or a simple formatting logic for `format_content`) using the extracted parameters. It has direct access to application state like the `vector_store` object.
    *   **Handle the Result:**
        *   *Note:* The provided example implementation **does not** seem to use the `ReturnControl` invocation type to send the function's result *back* to the *same* agent invocation to continue the turn.
        *   Instead, it executes the function, gets the result (e.g., `search_results`), formats it into a string, and effectively *ends* that agent invocation, returning the formatted string as the `result`.
        *   Subsequent steps (like formatting the research) involve a *new* call to `invoke_agent` for the writer agent, passing the result from the previous step as input. This differs from designs where the result is returned to the same agent turn for further reasoning.

## Pros

*   **Full Control:** The application has complete control over the execution environment and logic of the tools.
*   **Direct State Access:** Tools can directly access application memory, state, and resources (like the `vector_store` object in the example) without needing separate deployment or complex configuration passing.
*   **Simpler Local Development:** Can be easier to test and debug locally as the tool execution happens within the same process.
*   **Flexibility:** Allows integration with any library or service available to the application.

## Cons

*   **Application Burden:** The application code is responsible for implementing and executing the tool logic.
*   **Scalability:** The scalability of tool execution is tied to the scalability of the application itself.
*   **Tighter Coupling:** The agent's functionality is more tightly coupled with the application code.
*   **Interaction Model:** The specific implementation shown requires chaining separate agent invocations rather than letting the agent continue processing within a single turn after a tool is used. Implementing the latter (returning results via `ReturnControl`) adds complexity to the application's handling of the `invoke_agent` response/request cycle. 

## Setup and Configuration

First, let's import the necessary libraries and set up our environment:

In [1]:
import json
import logging
import os
import time
import uuid
from datetime import timedelta

import boto3
from couchbase.auth import PasswordAuthenticator
from couchbase.cluster import Cluster
from couchbase.exceptions import (InternalServerFailureException,
                                  QueryIndexAlreadyExistsException,
                                  ServiceUnavailableException)
from couchbase.management.buckets import CreateBucketSettings
from couchbase.management.search import SearchIndex
from couchbase.options import ClusterOptions
from dotenv import load_dotenv
from langchain_aws import BedrockEmbeddings
from langchain_couchbase.vectorstores import CouchbaseVectorStore

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

### Load Environment Variables

Load environment variables from the .env file. Make sure to create a .env file with the necessary credentials before running this notebook.

In [2]:
# Load environment variables
load_dotenv()

# Couchbase Configuration
CB_HOST = os.getenv("CB_HOST", "couchbase://localhost")
CB_USERNAME = os.getenv("CB_USERNAME", "Administrator")
CB_PASSWORD = os.getenv("CB_PASSWORD", "password")
CB_BUCKET_NAME = os.getenv("CB_BUCKET_NAME", "vector-search-testing")
SCOPE_NAME = os.getenv("SCOPE_NAME", "shared")
COLLECTION_NAME = os.getenv("COLLECTION_NAME", "bedrock") 
INDEX_NAME = os.getenv("INDEX_NAME", "vector_search_bedrock")

# AWS Configuration
AWS_REGION = os.getenv("AWS_REGION", "us-east-1")
AWS_ACCESS_KEY_ID = os.getenv("AWS_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY = os.getenv("AWS_SECRET_ACCESS_KEY")
AWS_ACCOUNT_ID = os.getenv("AWS_ACCOUNT_ID")

# Check if required environment variables are set
required_vars = ["AWS_ACCESS_KEY_ID", "AWS_SECRET_ACCESS_KEY"]
missing_vars = [var for var in required_vars if not os.getenv(var)]
if missing_vars:
    print(f"Missing required environment variables: {', '.join(missing_vars)}")
    print("Please set these variables in your .env file")
else:
    print("All required environment variables are set")

All required environment variables are set


### Initialize AWS Clients

Set up the AWS clients for Bedrock and other services:

In [3]:
# Initialize AWS session
session = boto3.Session(
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    region_name=AWS_REGION
)

# Initialize AWS clients from session
bedrock_client = session.client('bedrock')
bedrock_agent_client = session.client('bedrock-agent')
bedrock_runtime = session.client('bedrock-runtime')
bedrock_runtime_client = session.client('bedrock-agent-runtime')

print("AWS clients initialized successfully")

AWS clients initialized successfully


## Set Up Couchbase and Vector Store

Now let's set up the Couchbase connection, collections, and vector store:

In [4]:
def setup_collection(cluster, bucket_name, scope_name, collection_name):
    """Set up Couchbase collection"""
    try:
        # Check if bucket exists, create if it doesn't
        try:
            bucket = cluster.bucket(bucket_name)
            print(f"Bucket '{bucket_name}' exists.")
        except Exception as e:
            print(f"Bucket '{bucket_name}' does not exist. Creating it...")
            bucket_settings = CreateBucketSettings(
                name=bucket_name,
                bucket_type='couchbase',
                ram_quota_mb=1024,
                flush_enabled=True,
                num_replicas=0
            )
            cluster.buckets().create_bucket(bucket_settings)
            bucket = cluster.bucket(bucket_name)
            print(f"Bucket '{bucket_name}' created successfully.")

        bucket_manager = bucket.collections()

        # Check if scope exists, create if it doesn't
        scopes = bucket_manager.get_all_scopes()
        scope_exists = any(scope.name == scope_name for scope in scopes)
        
        if not scope_exists and scope_name != "_default":
            print(f"Scope '{scope_name}' does not exist. Creating it...")
            bucket_manager.create_scope(scope_name)
            print(f"Scope '{scope_name}' created successfully.")

        # Check if collection exists, create if it doesn't
        collections = bucket_manager.get_all_scopes()
        collection_exists = any(
            scope.name == scope_name and collection_name in [col.name for col in scope.collections]
            for scope in collections
        )

        if not collection_exists:
            print(f"Collection '{collection_name}' does not exist. Creating it...")
            bucket_manager.create_collection(scope_name, collection_name)
            print(f"Collection '{collection_name}' created successfully.")
        else:
            print(f"Collection '{collection_name}' already exists. Skipping creation.")

        # Wait for collection to be ready
        collection = bucket.scope(scope_name).collection(collection_name)
        time.sleep(2)  # Give the collection time to be ready for queries

        # Ensure primary index exists
        try:
            cluster.query(f"CREATE PRIMARY INDEX IF NOT EXISTS ON `{bucket_name}`.`{scope_name}`.`{collection_name}`").execute()
            print("Primary index present or created successfully.")
        except Exception as e:
            print(f"Error creating primary index: {str(e)}")

        # Clear all documents in the collection
        try:
            query = f"DELETE FROM `{bucket_name}`.`{scope_name}`.`{collection_name}`"
            cluster.query(query).execute()
            print("All documents cleared from the collection.")
        except Exception as e:
            print(f"Error while clearing documents: {str(e)}. The collection might be empty.")

        return collection
    except Exception as e:
        print(f"Error setting up collection: {str(e)}")
        raise

In [5]:
def setup_indexes(cluster):
    """Set up search indexes"""
    try:
        # Load index definition from file
        with open('aws_index.json', 'r') as file:
            index_definition = json.load(file)
            print(f"Loaded index definition from aws_index.json")
    except Exception as e:
        print(f"Error loading index definition: {str(e)}")
        raise
    
    try:
        scope_index_manager = cluster.bucket(CB_BUCKET_NAME).scope(SCOPE_NAME).search_indexes()

        # Check if index already exists
        existing_indexes = scope_index_manager.get_all_indexes()
        index_name = index_definition["name"]

        if index_name in [index.name for index in existing_indexes]:
            print(f"Index '{index_name}' found")
        else:
            print(f"Creating new index '{index_name}'...")

        # Create SearchIndex object from JSON definition
        search_index = SearchIndex.from_json(index_definition)

        # Upsert the index (create if not exists, update if exists)
        scope_index_manager.upsert_index(search_index)
        print(f"Index '{index_name}' successfully created/updated.")

    except QueryIndexAlreadyExistsException:
        print(f"Index '{index_name}' already exists. Skipping creation/update.")
    except ServiceUnavailableException:
        print("Search service is not available. Please ensure the Search service is enabled in your Couchbase cluster.")
    except InternalServerFailureException as e:
        print(f"Internal server error: {str(e)}")
        raise

In [6]:
# Connect to Couchbase
auth = PasswordAuthenticator(CB_USERNAME, CB_PASSWORD)
options = ClusterOptions(auth)
cluster = Cluster(CB_HOST, options)
cluster.wait_until_ready(timedelta(seconds=5))
print("Successfully connected to Couchbase")

# Set up collections
collection = setup_collection(cluster, CB_BUCKET_NAME, SCOPE_NAME, COLLECTION_NAME)
print("Collections setup complete")

# Set up search indexes
setup_indexes(cluster)
print("Search indexes setup complete")

Successfully connected to Couchbase
Bucket 'vector-search-testing' exists.
Collection 'bedrock' already exists. Skipping creation.
Primary index present or created successfully.
All documents cleared from the collection.
Collections setup complete
Loaded index definition from aws_index.json
Index 'vector_search_bedrock' found
Index 'vector_search_bedrock' already exists. Skipping creation/update.
Search indexes setup complete


In [7]:
# Initialize Bedrock runtime client for embeddings
embeddings = BedrockEmbeddings(
    client=bedrock_runtime,
    model_id="amazon.titan-embed-text-v2:0"
)
print("Successfully created Bedrock embeddings client")

# Initialize vector store
vector_store = CouchbaseVectorStore(
    cluster=cluster,
    bucket_name=CB_BUCKET_NAME,
    scope_name=SCOPE_NAME,
    collection_name=COLLECTION_NAME,
    embedding=embeddings,
    index_name=INDEX_NAME
)
print("Successfully created vector store")

Successfully created Bedrock embeddings client
Successfully created vector store


## Helper Functions for Vector Store Operations

Let's define some helper functions for working with the vector store:

In [8]:
def add_document(vector_store, text, metadata=None):
    """Add a document to the vector store"""
    if metadata is None:
        metadata = {}
    elif isinstance(metadata, str):
        try:
            metadata = json.loads(metadata)
        except json.JSONDecodeError:
            metadata = {}
    return vector_store.add_texts([text], [metadata])[0]

def search_documents(vector_store, query, k=4):
    """Search for similar documents"""
    return vector_store.similarity_search(query, k=k)

## Load Documents from JSON File

Let's load the documents from the documents.json file:

In [9]:
# Load documents from JSON file
try:
    with open('documents.json', 'r') as f:
        data = json.load(f)
        documents = data.get('documents', [])
    print(f"Loaded {len(documents)} documents from documents.json")
except Exception as e:
    print(f"Error loading documents: {str(e)}")
    raise

# Add documents to vector store
print(f"Adding {len(documents)} documents to vector store...")
for i, doc in enumerate(documents, 1):
    text = doc.get('text', '')
    metadata = doc.get('metadata', {})
    
    # Add document to vector store
    doc_id = add_document(vector_store, text, json.dumps(metadata))
    print(f"Added document {i}/{len(documents)} with ID: {doc_id}")
    
    # Add small delay between requests
    time.sleep(1)

print(f"\nProcessing complete: {len(documents)}/{len(documents)} documents added successfully")

Loaded 7 documents from documents.json
Adding 7 documents to vector store...
Added document 1/7 with ID: c6f964f0a6d94db78f47dcd068af9deb
Added document 2/7 with ID: 920b7b9e20354b7a8e4250ce99b57d04
Added document 3/7 with ID: 20cb23e471e846cfb1aca71d1fdd821d
Added document 4/7 with ID: d626656869e545cfa08cce1a7873d7eb
Added document 5/7 with ID: ffcae2962cce4dd9b7efce77ec1c8c42
Added document 6/7 with ID: 5e06b6355d124519a7af21d82a001e72
Added document 7/7 with ID: 611abe851abe44dcaa7a6d20d580fffa

Processing complete: 7/7 documents added successfully


## Custom Control Approach Implementation

Now let's implement the Custom Control approach for Bedrock agents:

In [10]:
def wait_for_agent_status(bedrock_agent_client, agent_id, target_statuses=['Available', 'PREPARED', 'NOT_PREPARED'], max_attempts=30, delay=2):
    """Wait for agent to reach any of the target statuses"""
    for attempt in range(max_attempts):
        try:
            response = bedrock_agent_client.get_agent(agentId=agent_id)
            current_status = response['agent']['agentStatus']
            
            if current_status in target_statuses:
                print(f"Agent {agent_id} reached status: {current_status}")
                return current_status
            elif current_status == 'FAILED':
                print(f"Agent {agent_id} failed")
                return 'FAILED'
            
            print(f"Agent status: {current_status}, waiting... (attempt {attempt + 1}/{max_attempts})")
            time.sleep(delay)
            
        except Exception as e:
            print(f"Error checking agent status: {str(e)}")
            time.sleep(delay)
    
    return current_status

In [11]:
def create_agent(bedrock_agent_client, name, instructions, functions, model_id="amazon.nova-pro-v1:0"):
    """Create a Bedrock agent with Custom Control action groups"""
    try:
        # List existing agents
        existing_agents = bedrock_agent_client.list_agents()
        existing_agent = next(
            (agent for agent in existing_agents['agentSummaries'] 
             if agent['agentName'] == name),
            None
        )
        
        # Handle existing agent
        if existing_agent:
            agent_id = existing_agent['agentId']
            print(f"Found existing agent '{name}' with ID: {agent_id}")
            
            # Check agent status
            response = bedrock_agent_client.get_agent(agentId=agent_id)
            status = response['agent']['agentStatus']
            
            if status in ['NOT_PREPARED', 'FAILED']:
                print(f"Deleting agent '{name}' with status {status}")
                bedrock_agent_client.delete_agent(agentId=agent_id)
                time.sleep(10)  # Wait after deletion
                existing_agent = None
        
        # Create new agent if needed
        if not existing_agent:
            print(f"Creating new agent '{name}'")
            agent = bedrock_agent_client.create_agent(
                agentName=name,
                description=f"{name.title()} agent for document operations",
                instruction=instructions,
                idleSessionTTLInSeconds=1800,
                foundationModel=model_id
            )
            agent_id = agent['agent']['agentId']
            print(f"Created new agent '{name}' with ID: {agent_id}")
        else:
            agent_id = existing_agent['agentId']
        
        # Wait for initial creation if needed
        status = wait_for_agent_status(bedrock_agent_client, agent_id, target_statuses=['NOT_PREPARED', 'PREPARED', 'Available'])
        if status not in ['NOT_PREPARED', 'PREPARED', 'Available']:
            raise Exception(f"Agent failed to reach valid state: {status}")
        
        # Create action group if needed
        try:
            bedrock_agent_client.create_agent_action_group(
                agentId=agent_id,
                agentVersion="DRAFT",
                actionGroupExecutor={"customControl": "RETURN_CONTROL"},  # This is the key for Custom Control
                actionGroupName=f"{name}_actions",
                functionSchema={"functions": functions},
                description=f"Action group for {name} operations"
            )
            print(f"Created action group for agent '{name}'")
            time.sleep(5)
        except bedrock_agent_client.exceptions.ConflictException:
            print(f"Action group already exists for agent '{name}'")
        
        # Prepare agent if needed
        if status == 'NOT_PREPARED':
            try:
                print(f"Starting preparation for agent '{name}'")
                bedrock_agent_client.prepare_agent(agentId=agent_id)
                status = wait_for_agent_status(
                    bedrock_agent_client,
                    agent_id, 
                    target_statuses=['PREPARED', 'Available']
                )
                print(f"Agent '{name}' preparation completed with status: {status}")
            except Exception as e:
                print(f"Error during preparation: {str(e)}")
        
        # Handle alias creation/retrieval
        try:
            aliases = bedrock_agent_client.list_agent_aliases(agentId=agent_id)
            alias = next((a for a in aliases['agentAliasSummaries'] if a['agentAliasName'] == 'v1'), None)
            
            if not alias:
                print(f"Creating new alias for agent '{name}'")
                alias = bedrock_agent_client.create_agent_alias(
                    agentId=agent_id,
                    agentAliasName="v1"
                )
                alias_id = alias['agentAlias']['agentAliasId']
            else:
                alias_id = alias['agentAliasId']
                print(f"Using existing alias for agent '{name}'")
            
            print(f"Successfully configured agent '{name}' with ID: {agent_id} and alias: {alias_id}")
            return agent_id, alias_id
            
        except Exception as e:
            print(f"Error managing alias: {str(e)}")
            raise
        
    except Exception as e:
        print(f"Error creating/updating agent: {str(e)}")
        raise

In [12]:
def invoke_agent(bedrock_runtime_client, agent_id, alias_id, input_text, session_id=None, vector_store=None):
    """Invoke a Bedrock agent"""
    if session_id is None:
        session_id = str(uuid.uuid4())
        
    try:
        print(f"Invoking agent with input: {input_text}")
        
        response = bedrock_runtime_client.invoke_agent(
            agentId=agent_id,
            agentAliasId=alias_id,
            sessionId=session_id,
            inputText=input_text,
            enableTrace=True
        )
        
        result = ""
        
        for i, event in enumerate(response['completion']):
            # Process text chunks
            if 'chunk' in event:
                chunk = event['chunk']['bytes'].decode('utf-8')
                result += chunk
            
            # Process trace data if needed
            if 'trace' in event:
                trace_data = event['trace']
                
                if isinstance(trace_data, dict) and 'orchestrationTrace' in trace_data:
                    orch_trace = trace_data['orchestrationTrace']
                    
                    if 'invocationOutput' in orch_trace:
                        invocation_output = orch_trace['invocationOutput']
                        
                        if 'actionGroupInvocationOutput' in invocation_output:
                            action_output = invocation_output['actionGroupInvocationOutput']
                            
                            if 'responseBody' in action_output:
                                response_body = action_output['responseBody']
                                
                                if isinstance(response_body, dict) and 'application/json' in response_body:
                                    json_body = response_body['application/json']
                                    
                                    if 'body' in json_body:
                                        lambda_result = json_body['body']
                                        result = lambda_result
            
            # Handle custom control return
            if 'returnControl' in event:
                return_control = event['returnControl']
                invocation_inputs = return_control.get('invocationInputs', [])
                
                if invocation_inputs:
                    function_input = invocation_inputs[0].get('functionInvocationInput', {})
                    action_group = function_input.get('actionGroup')
                    function_name = function_input.get('function')
                    parameters = function_input.get('parameters', [])
                    
                    # Convert parameters to a dictionary
                    param_dict = {}
                    for param in parameters:
                        param_dict[param.get('name')] = param.get('value')
                    
                    print(f"Function call: {action_group}::{function_name}")
                    
                    # Handle search_documents function
                    if function_name == 'search_documents':
                        query = param_dict.get('query')
                        k = int(param_dict.get('k', 3))
                        
                        print(f"Searching for: {query}, k={k}")
                        
                        if vector_store:
                            # Perform the search
                            docs = search_documents(vector_store, query, k)
                            
                            # Format results
                            search_results = [doc.page_content for doc in docs]
                            print(f"Found {len(search_results)} results")
                            
                            # Format the response
                            result = f"Search results for '{query}':\n\n"
                            for i, content in enumerate(search_results):
                                result += f"Result {i+1}: {content}\n\n"
                        else:
                            print("Vector store not available")
                            result = "Error: Vector store not available"
                    
                    # Handle format_content function
                    elif function_name == 'format_content':
                        content = param_dict.get('content')
                        style = param_dict.get('style', 'user-friendly')
                        
                        print(f"Formatting content in {style} style")
                        
                        # Check if content is valid
                        if content and content != '?':
                            result = f"Formatted in {style} style: {content}"
                        else:
                            result = "No content provided to format."
                    else:
                        print(f"Unknown function: {function_name}")
                        result = f"Error: Unknown function {function_name}"
        
        if not result.strip():
            print("Received empty response from agent")
        
        return result
        
    except Exception as e:
        print(f"Error invoking agent: {str(e)}")
        raise RuntimeError(f"Failed to invoke agent: {str(e)}")

## Define Agent Instructions and Functions

Now let's define the instructions and functions for our agents:

In [13]:
# Researcher agent instructions
researcher_instructions = """
You are a Research Assistant that helps users find relevant information in documents.
Your capabilities include:
1. Searching through documents using semantic similarity
2. Providing relevant document excerpts
3. Answering questions based on document content
"""

# Researcher agent functions
researcher_functions = [{
    "name": "search_documents",
    "description": "Search for relevant documents using semantic similarity",
    "parameters": {
        "query": {
            "type": "string",
            "description": "The search query",
            "required": True
        },
        "k": {
            "type": "integer",
            "description": "Number of results to return",
            "required": False
        }
    },
    "requireConfirmation": "DISABLED"
}]

# Writer agent instructions
writer_instructions = """
You are a Content Writer Assistant that helps format and present research findings.
Your capabilities include:
1. Formatting research findings in a user-friendly way
2. Creating clear and engaging summaries
3. Organizing information logically
4. Highlighting key insights
"""

# Writer agent functions
writer_functions = [{
    "name": "format_content",
    "description": "Format and present research findings",
    "parameters": {
        "content": {
            "type": "string",
            "description": "The research findings to format",
            "required": True
        },
        "style": {
            "type": "string",
            "description": "The desired presentation style (e.g., summary, detailed, bullet points)",
            "required": False
        }
    },
    "requireConfirmation": "DISABLED"
}]

## Run Custom Control Approach

Now let's run the Custom Control approach with our agents:

In [14]:
# Create researcher agent
try:
    researcher_id, researcher_alias = create_agent(
        bedrock_agent_client,
        "researcher", 
        researcher_instructions, 
        researcher_functions
    )
    print(f"Researcher agent created with ID: {researcher_id} and alias: {researcher_alias}")
except Exception as e:
    print(f"Failed to create researcher agent: {str(e)}")
    researcher_id, researcher_alias = None, None

# Create writer agent
try:
    writer_id, writer_alias = create_agent(
        bedrock_agent_client,
        "writer", 
        writer_instructions, 
        writer_functions
    )
    print(f"Writer agent created with ID: {writer_id} and alias: {writer_alias}")
except Exception as e:
    print(f"Failed to create writer agent: {str(e)}")
    writer_id, writer_alias = None, None

if not any([researcher_id, writer_id]):
    raise RuntimeError("Failed to create any agents")

Found existing agent 'researcher' with ID: FZX7XPEHVK
Agent FZX7XPEHVK reached status: PREPARED
Action group already exists for agent 'researcher'
Using existing alias for agent 'researcher'
Successfully configured agent 'researcher' with ID: FZX7XPEHVK and alias: 8A60QDPM05
Researcher agent created with ID: FZX7XPEHVK and alias: 8A60QDPM05
Found existing agent 'writer' with ID: J08JUS1UVN
Agent J08JUS1UVN reached status: PREPARED
Action group already exists for agent 'writer'
Using existing alias for agent 'writer'
Successfully configured agent 'writer' with ID: J08JUS1UVN and alias: YJKVWNCWRL
Writer agent created with ID: J08JUS1UVN and alias: YJKVWNCWRL


## Test the Agents

Let's test our agents by asking the researcher agent to search for information and the writer agent to format the results:

In [15]:
# Test researcher agent
researcher_response = invoke_agent(
    bedrock_runtime_client,
    researcher_id,
    researcher_alias,
    'What is unique about the Cline AI assistant? Use the search_documents function to find relevant information.',
    vector_store=vector_store
)
print("\nResearcher Response:\n", researcher_response)

Invoking agent with input: What is unique about the Cline AI assistant? Use the search_documents function to find relevant information.
Function call: researcher_actions::search_documents
Searching for: Cline AI assistant unique features, k=5
Found 5 results

Researcher Response:
 Search results for 'Cline AI assistant unique features':

Result 1: The Cline AI assistant, developed by Saoud Rizwan, is a unique system that combines vector search capabilities with Amazon Bedrock agents. Unlike traditional chatbots, it uses a sophisticated multi-agent architecture where specialized agents handle different aspects of document processing and interaction.

Result 2: One of Cline's key features is its ability to create MCP (Model Context Protocol) servers on the fly. This allows users to extend the system's capabilities by adding new tools and resources that connect to external APIs, all while maintaining a secure and non-interactive environment.

Result 3: The browser automation capabilities 

In [16]:
# Test writer agent
writer_response = invoke_agent(
    bedrock_runtime_client,
    writer_id,
    writer_alias,
    f'Format this research finding using the format_content function: {researcher_response}',
    vector_store=vector_store
)
print("\nWriter Response:\n", writer_response)

Invoking agent with input: Format this research finding using the format_content function: Search results for 'Cline AI assistant unique features':

Result 1: The Cline AI assistant, developed by Saoud Rizwan, is a unique system that combines vector search capabilities with Amazon Bedrock agents. Unlike traditional chatbots, it uses a sophisticated multi-agent architecture where specialized agents handle different aspects of document processing and interaction.

Result 2: One of Cline's key features is its ability to create MCP (Model Context Protocol) servers on the fly. This allows users to extend the system's capabilities by adding new tools and resources that connect to external APIs, all while maintaining a secure and non-interactive environment.

Result 3: The browser automation capabilities in Cline are implemented through Puppeteer, allowing the system to interact with web interfaces in a controlled 900x600 pixel window. This enables testing of web applications, verification of

## Conclusion

In this notebook, we've demonstrated the Custom Control approach for implementing AWS Bedrock agents with Couchbase Vector Search. This approach allows the agent to return control to the application for function execution, providing more flexibility and control over the agent's behavior.

Key components of this implementation include:

1. **Vector Store Setup**: We set up a Couchbase vector store to store and search documents using semantic similarity.
2. **Agent Creation**: We created two specialized agents - a researcher agent for searching documents and a writer agent for formatting results.
3. **Custom Control**: We implemented the Custom Control approach, where the agent returns control to the application for function execution.
4. **Function Handling**: We handled the agent's function calls in the application code, allowing for more control and flexibility.

This approach is particularly useful when you need more control over the agent's behavior or when you want to integrate the agent with existing systems and data sources.